In [5]:
import os, glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from scipy.signal import savgol_filter, find_peaks, peak_widths
from scipy import integrate
from scipy.sparse import diags, eye as sp_eye
from scipy.sparse.linalg import spsolve
from typing import List, Tuple, Dict, Optional

# ===== Baselines =====
def _poly_baseline(x, y, deg=3):
    if len(x) < deg + 1:
        return np.zeros_like(y)
    mask = np.ones_like(y, dtype=bool)
    for _ in range(3):
        coeffs = np.polyfit(x[mask], y[mask], deg)
        base = np.polyval(coeffs, x)
        mask = y <= base
    return np.polyval(np.polyfit(x, y, deg), x)

def _als_baseline_sparse(y, lam=1e6, p=0.01, niter=10):
    """Asymmetric least squares baseline (sparse, robust for long signals)."""
    L = len(y)
    if L < 3:
        return np.zeros_like(y)
    # second-difference operator (L-2 by L) as sparse:
    D = diags([1, -2, 1], [0, -1, -2], shape=(L-2, L))
    w = np.ones(L)
    for _ in range(niter):
        W = diags(w, 0, shape=(L, L))
        Z = W + lam * (D.T @ D)
        b = spsolve(Z, w * y)
        w = p * (y > b) + (1 - p) * (y < b)
    return b

def integrate_region(x, y, xlo, xhi):
    m = (x >= xlo) & (x <= xhi)
    if np.count_nonzero(m) < 2:
        return 0.0
    return integrate.trapezoid(y[m], x[m])

# ===== Core analysis =====
def analyze_peaks_in_regions(
    x: np.ndarray,
    y: np.ndarray,
    regions: List[Tuple[float, float]],
    smoothing: Optional[Dict] = None,
    baseline: Dict = None,
    min_prominence: float = None,
    max_peaks_per_region: int = 3,
) -> pd.DataFrame:
    if smoothing is None:
        smoothing = {'method':'savgol', 'window':15, 'polyorder':3}
    if baseline is None:
        baseline = {'method':'als', 'lam':1e6, 'p':0.01}

    x = np.asarray(x, float)
    y = np.asarray(y, float)

    # light smoothing (guard window length)
    if smoothing.get('method','savgol') == 'savgol':
        w = int(smoothing.get('window', 15))
        p = int(smoothing.get('polyorder', 3))
        w = max(5, w + (w % 2 == 0))  # odd, >=5
        # If global data are short, clamp window
        w = min(w, (len(x)//2)*2 - 1) if len(x) > 7 else 5
        y_s = savgol_filter(y, window_length=max(5, w), polyorder=min(p, 3), mode='interp')
    else:
        y_s = y.copy()

    out = []
    for r_id, (xlo, xhi) in enumerate(regions, start=1):
        m = (x >= xlo) & (x <= xhi)
        if np.count_nonzero(m) < 7:   # need a few points for widths
            continue

        xr = x[m]
        yr = y_s[m]

        # choose baseline per region
        bmethod = baseline.get('method','als')
        if bmethod == 'als':
            b = _als_baseline_sparse(yr, lam=baseline.get('lam', 1e6), p=baseline.get('p', 0.01))
        elif bmethod == 'poly':
            b = _poly_baseline(xr, yr, deg=baseline.get('deg', 3))
        else:
            b = np.zeros_like(yr)

        yr_corr = yr - b

        # auto-prominence if not provided
        prom = min_prominence
        if prom is None:
            mad = np.median(np.abs(yr_corr - np.median(yr_corr)))
            noise = 1.4826 * mad
            prom = max(noise * 5, (yr_corr.max() - yr_corr.min()) * 0.02)

        pks, props = find_peaks(yr_corr, prominence=prom)
        if pks.size == 0:
            continue

        # FWHM in sample-index units
        widths, w_left, w_right, _ = peak_widths(yr_corr, pks, rel_height=0.5)

        # Convert index positions to x by interpolation
        idx_axis = np.arange(len(xr), dtype=float)
        left_x = np.interp(w_left, idx_axis, xr)
        right_x = np.interp(w_right, idx_axis, xr)

        # FWHM in x-units
        fwhm_x = right_x - left_x

        # keep strongest peaks per region
        order = np.argsort(props['prominences'])[::-1][:max_peaks_per_region]

        for idx in order:
            i = pks[idx]
            peak_x = xr[i]
            # we'll write peak_y in raw coordinates (not baseline-corrected) later in process_folder
            height = yr_corr[i]  # above baseline
            area = integrate_region(xr, yr_corr, left_x[idx], right_x[idx])

            out.append({
                'region_id': r_id,
                'x_lo': xlo, 'x_hi': xhi,
                'peak_x': float(peak_x),
                'peak_y': float(yr[i]),            # smoothed signal at summit (temporary)
                'baseline_at_peak': float(b[i]),
                'height': float(height),
                'fwhm_x': float(fwhm_x[idx]),
                'left_ips_x': float(left_x[idx]),
                'right_ips_x': float(right_x[idx]),
                'area_under_peak': float(area),
                'prominence': float(props['prominences'][idx]),
            })

    return pd.DataFrame(out).sort_values(['region_id', 'peak_x']).reset_index(drop=True)

# ===== I/O + plotting =====
def load_xy(path):
    """Robustly read 2-column .xy with whitespace, comma, or semicolon."""
    try:
        data = np.loadtxt(path)
    except Exception:
        data = np.loadtxt(path, delimiter=',')
    if data.ndim != 2 or data.shape[1] < 2:
        raise ValueError(f"File {path} is not a 2-column XY.")
    return data[:,0], data[:,1]

def format_table_df(df: pd.DataFrame) -> pd.DataFrame:
    if df.empty:
        return pd.DataFrame({'info': ['No peaks found in regions.']})
    view = df[['region_id','peak_x','height','fwhm_x','area_under_peak']].copy()
    return (view
            .rename(columns={'region_id':'Reg','peak_x':'Position','height':'Height',
                             'fwhm_x':'FWHM','area_under_peak':'Area'})
            .astype({'Reg':int})
            .round({'Position':4, 'Height':2, 'FWHM':4, 'Area':2})
           )

def plot_trace_with_table(x, y, peaks_df, title=None, out_png=None):
    fig = plt.figure(figsize=(10, 6.5))
    gs = GridSpec(3, 1, height_ratios=[2.0, 0.05, 1.0])
    ax = fig.add_subplot(gs[0, 0])

    ax.plot(x, y, lw=1)
    ax.set_xlabel(r'$2\theta$')
    ax.set_ylabel('Intensity (a.u.)')
    if title:
        ax.set_title(title)

    if not peaks_df.empty:
        ax.scatter(peaks_df['peak_x'], peaks_df['peak_y_raw'], s=35, marker='x')
        for _, r in peaks_df.iterrows():
            ax.annotate(f"{r['peak_x']:.3f}",
                        xy=(r['peak_x'], r['peak_y_raw']),
                        xytext=(0, 6), textcoords='offset points',
                        ha='center', va='bottom', fontsize=8)

    ax_tbl = fig.add_subplot(gs[2, 0])
    ax_tbl.axis('off')
    tbl_df = format_table_df(peaks_df)
    table = ax_tbl.table(cellText=tbl_df.values,
                         colLabels=tbl_df.columns.tolist(),
                         loc='center')
    table.auto_set_font_size(False)
    table.set_fontsize(9)
    table.scale(1, 1.2)

    fig.tight_layout()
    if out_png:
        fig.savefig(out_png, dpi=200, bbox_inches='tight')
    plt.close(fig)

def process_folder(
    folder: str,
    regions: List[Tuple[float, float]],
    smoothing={'method':'savgol','window':21,'polyorder':3},
    baseline={'method':'als','lam':1e5,'p':0.01},
    min_prominence=None,
    max_peaks_per_region=2,
    output_dir='peak_reports'
):
    os.makedirs(output_dir, exist_ok=True)
    files = sorted(glob.glob(os.path.join(folder, "*.xy")))
    if not files:
        print("No .xy files found.")
        return

    all_rows = []
    for f in files:
        try:
            x, y = load_xy(f)
        except Exception as e:
            print(f"Skipping {os.path.basename(f)}: {e}")
            continue

        df = analyze_peaks_in_regions(
            x, y, regions, smoothing, baseline,
            min_prominence=min_prominence,
            max_peaks_per_region=max_peaks_per_region
        )

        if not df.empty:
            # Replace temporary peak_y with RAW y at nearest index (prettier on plot)
            idxs = np.searchsorted(x, df['peak_x'].values)
            idxs = np.clip(idxs, 0, len(x)-1)
            df['peak_y_raw'] = y[idxs]
            # keep smoothed value too if you like:
            # df.rename(columns={'peak_y':'peak_y_smoothed'}, inplace=True)
            df.insert(0, 'file', os.path.splitext(os.path.basename(f))[0])

        base = os.path.splitext(os.path.basename(f))[0]
        out_png = os.path.join(output_dir, f"{base}_peaks.png")
        plot_trace_with_table(x, y, df if not df.empty else pd.DataFrame(), title=base, out_png=out_png)

        if not df.empty:
            all_rows.append(df)

    if all_rows:
        summary = pd.concat(all_rows, ignore_index=True)
        summary.to_csv(os.path.join(output_dir, "peak_summary.csv"), index=False)
        print(f"Saved figures + summary CSV in: {os.path.abspath(output_dir)}")
    else:
        print("No peaks were found in the specified regions for any file.")

# ===== Example run (edit paths/regions) =====
if __name__ == "__main__":
    FOLDER = r"C:\\Dilan\\Study\\Master Thesis\\Characterization\\XRD\\30-09-2025 Dep No 1\\XY Data"
    REGIONS = [
        (20.5, 23.0),
        (30.5, 33.0),
        (37.5, 40.0),
        (45.0, 47.5),
        (50.0, 52.5),
        (55.0, 57.5),
    ]
    process_folder(FOLDER, REGIONS)


C:\Users\Dilan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\scipy\sparse\linalg\_dsolve\linsolve.py:293: MatrixRankWarning: Matrix is exactly singular
  warn("Matrix is exactly singular", MatrixRankWarning)


KeyError: 'region_id'